In [ ]:
import pandas as pd
import transformers
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

# load only a sub-selection of the categories (3 in our case)
categories = ['alt.atheism', 'comp.graphics', 'sci.space']

# remove the headers, footers and quotes (to avoid overfitting)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=categories, random_state=42)
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=categories, random_state=42)

train = pd.DataFrame({'text': newsgroups_train.data, 'labels': newsgroups_train.target})
test = pd.DataFrame({'text': newsgroups_test.data, 'labels': newsgroups_test.target})

train, dev = train_test_split(train, test_size=0.1, random_state=0,
                               stratify=train[['labels']])

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = Dataset.from_pandas(train)
dev_dataset = Dataset.from_pandas(dev)
test_dataset = Dataset.from_pandas(test)

train_dataset = train_dataset.map(tokenize_function, batched=True)
dev_dataset = dev_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/1491 [00:00<?, ? examples/s]

Map:   0%|          | 0/166 [00:00<?, ? examples/s]

Map:   0%|          | 0/1102 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=3)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

results = trainer.evaluate(test_dataset)
print(results)

Checkpoint destination directory ./results/checkpoint-187 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-374 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-561 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-748 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-935 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'eval_loss': 0.5312819480895996, 'eval_runtime': 31.6244, 'eval_samples_per_second': 34.846, 'eval_steps_per_second': 4.364, 'epoch': 5.0}


In [ ]:
from sklearn.metrics import classification_report
predictions = trainer.predict(test_dataset)

In [ ]:
import numpy as np

pred_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

report = classification_report(true_labels, pred_labels, target_names=categories)
print(report)

               precision    recall  f1-score   support

  alt.atheism       0.88      0.88      0.88       319
comp.graphics       0.91      0.95      0.93       389
    sci.space       0.89      0.85      0.87       394

     accuracy                           0.89      1102
    macro avg       0.89      0.89      0.89      1102
 weighted avg       0.89      0.89      0.89      1102



# **roBERTa model**

The model has high accuracy with balanced precision and recall. The scores suggest that the model identifies instances with high accuracy but also keeps a lows rate of FN. F1-scores indicate that the model is robust and performs well across different types of text data.

The process of fine-tuning the Roberta model has proven to be effective. We would also suggest that it could be improved even more, but since Collab has some memory restrictions, we were not able to check it.

Also we noticed the the evaluation loss start to increase with the time. Probably thats because of the overfitting, but we were unable to fix it.

Overall roBERTa exhibit strong performance across various metrics.




In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

svm_model = SVC(kernel='linear')
svm_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', svm_model)
])

svm_pipeline.fit(train['text'], train['labels'])
svm_predictions = svm_pipeline.predict(test['text'])

svm_report = classification_report(test['labels'], svm_predictions, target_names=categories)
print(svm_report)

               precision    recall  f1-score   support

  alt.atheism       0.87      0.78      0.82       319
comp.graphics       0.91      0.85      0.88       389
    sci.space       0.79      0.90      0.84       394

     accuracy                           0.85      1102
    macro avg       0.85      0.85      0.85      1102
 weighted avg       0.85      0.85      0.85      1102



# **Overall**
Both BERT and RoBERTa models show superior performance over the conventional SVM approach in terms of precision, recall, F1-score, and overall accuracy.

The identical scores between BERT and roBERTa most probably because of the fact that the latter one builds on the first one.